In [1]:
import pandas as pd
pd.options.display.max_columns = 200

# Import Dati Sorgente
Vengono importate le seguenti sorgenti dati:
1. CUP con feedback dal DIPE e relativi template associati
2. CUP scaricati da OpenCoesione [(vedi qui - link)](https://opencoesione.gov.it/it/progetti/?q=&selected_facets=is_pubblicato:0&selected_facets=is_pubblicato:1&selected_facets=is_pubblicato:2)

In [2]:
#  xls_DIPE = pd.ExcelFile('../result/CUP_feedback.xlsx')
xls_DIPE = pd.ExcelFile('../data/01_raw/Tutti i CUP al 20230322.xlsx')
df = pd.read_excel(xls_DIPE, 'CUP ').iloc[:-2,:]
print(df.info())
df.to_parquet('../data/02_primary/CUP_feedback.xlsx')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60343 entries, 0 to 60342
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CODICE_CUP             60343 non-null  object        
 1   SOGGETTO_TITOLARE      60343 non-null  object        
 2   DATA_GENERAZIONE       60341 non-null  datetime64[ns]
 3   TEMPLATE               60343 non-null  float64       
 4   COPERTURA_FINANZIARIA  60343 non-null  object        
 5   COSTO                  60343 non-null  float64       
 6   FINANZIAMENTO          60343 non-null  float64       
 7   DESCRIZIONE_AGGREGATA  60343 non-null  object        
 8   NATURA                 60343 non-null  object        
 9   TIPOLOGIA              60343 non-null  object        
 10  SETTORE                60343 non-null  object        
 11  SOTTOSETTORE           60343 non-null  object        
 12  CATEGORIA              60343 non-null  object        
 13  S

In [3]:
# xls_PAD2026 = pd.ExcelFile('../data/01_raw/Lista CUP/lista dtd.xlsx')
# df_pad2026 = pd.read_excel(xls_PAD2026,'lista')
# df.to_csv('../data/02_primary/CUP_PAD2026.csv')

In [4]:

df_OC = pd.read_csv(
    '../data/01_raw/progetti_OC.zip',
    compression='zip', 
    sep=';',
    quotechar='"', 
    low_memory=False, 
    usecols=['COD_LOCALE_PROGETTO','CUP','OC_TITOLO_PROGETTO','OC_DESCR_CICLO','OC_COD_CICLO','OC_TEMA_SINTETICO','FONDO_COMUNITARIO','OC_CODICE_PROGRAMMA','OC_DESCRIZIONE_PROGRAMMA','COD_STRUMENTO','DESCR_STRUMENTO','OC_STATO_FINANZIARIO','OC_STATO_PROGETTO','OC_STATO_PROCEDURALE'],
    header=0,
    #nrows=10
    )

In [5]:
df_pad2026 = pd.read_csv('../data/01_raw/LISTA_CUP_SF.csv') # estratto direttamente da Salesforce tramite query
df_pad2026.columns = ['_','CODICE_CUP','__','AVVISO','STATO_CANDIDATURA']
df_pad2026.drop(columns=['_','__'],  inplace = True)
df_pad2026.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26575 entries, 0 to 26574
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CODICE_CUP         26574 non-null  object
 1   AVVISO             26575 non-null  object
 2   STATO_CANDIDATURA  26575 non-null  object
dtypes: object(3)
memory usage: 623.0+ KB


In [6]:
candidature_attive = df_pad2026.STATO_CANDIDATURA.isin(['ACCETTATA','FINANZIATA'])
CUP_DUPLICATI = df_pad2026[candidature_attive].CODICE_CUP.duplicated()
df_pad2026[candidature_attive][CUP_DUPLICATI]

,CODICE_CUP,AVVISO,STATO_CANDIDATURA
10787,D31F22004420006,"1.4.5 ""Piattaforma Notifiche Digitali” Comuni ...",FINANZIATA
20516,D91F22003340006,1.2 Abilitazione al Cloud - Comuni - Luglio 2022,FINANZIATA
21777,F91F22002170006,1.4.3 - pagoPA - Comuni - Aprile 2022,FINANZIATA


# Controlli da fare

Di seguito vengono fatti i seguenti controlli, in ordine di priorità:

1. CUP PNRR in OpenCoesione
2. CUP duplicato nel PNRR
3. CUP non esistente nei sistemi DIPE
4. CUP senza template
5. CUP template diverso


In [7]:
# concateno dataset da OpenCoesione (OC)
# df_OC = pd.concat([df_OC, df_OC2]).copy()

df_OC = df_OC.drop_duplicates(subset='CUP',keep='last').copy()

In [8]:
print(df_OC.CUP.nunique())

1274053


In [9]:
df_DIPE = pd.merge(df_pad2026,df, on = 'CODICE_CUP', how ='left')
df_DIPE.loc[df_DIPE.STATO_PROGETTO.isna(),'STATO_PROGETTO'] = 'CUP INESISTENTE'

In [10]:
df_DIPE['CUP'] = df_DIPE.CODICE_CUP

In [16]:
df_DIPE[df_DIPE.TEMPLATE.notna()].TEMPLATE

0        2203007.0
1        2203007.0
2        2203006.0
3        2208001.0
4        2208001.0
           ...    
26570    2203007.0
26571    2203006.0
26572    2210002.0
26573    2203006.0
26574    2203004.0
Name: TEMPLATE, Length: 25846, dtype: float64

In [19]:
# Primi check


## CUP double funding
DOUBLE_FUNDING_CUP = pd.merge( df_OC['CUP'], df_DIPE['CUP'], on = 'CUP',how='inner',validate="1:m").CUP.unique()
## CUP double project
DOUBLE_REQUEST_CUP = df_DIPE.groupby(by = 'CUP').filter(lambda x: len(x)>1).sort_values(by='CUP').CUP.unique()
## CUP non esistente nei sistemi DIPE
MISSING_CUP = df_DIPE[df_DIPE.STATO_PROGETTO=='CUP INESISTENTE'].CUP.unique()
## Template mancante
MISSING_TEMPLATE = df_DIPE[df_DIPE.TEMPLATE.isna()].CUP.unique()
#Template errato
map_sheet2template = df_DIPE[(df_DIPE.TEMPLATE.notna())].groupby(by='AVVISO').TEMPLATE.agg(pd.Series.mode).to_dict()
# map_sheet2template['1.4.4 Altri Enti FINESTRA_76'] = 20220511
# map_sheet2template['1.4.3 Altri Enti App IOFINESTRA'] = 20220509
# map_sheet2template[ '1.4.3 pagoPA Altri EntiFINESTRA'] = 20220510
WRONG_TEMPLATE = [] 
WRONG_TEMPLATE = df_DIPE[df_DIPE.AVVISO.replace(map_sheet2template) != df_DIPE.TEMPLATE].CUP.unique()

1094

In [25]:
def classify_CUP(CUP: str):
    '''
    Funzione per produrre una semplice classificazione dei CUP secondo una scala di priorità:
    1. CUP PNRR in OpenCoesione
    2. CUP duplicato nel PNRR
    3. CUP non esistente nei sistemi DIPE
    4. CUP senza template
    5. CUP template diverso
    '''

    if CUP in DOUBLE_FUNDING_CUP:
        return 'CUP già presente in OPEN COESIONE'
    elif CUP in MISSING_CUP:
        return 'CUP NON PRESENTE NEL FEEDBACK DIPE'
    elif CUP in DOUBLE_REQUEST_CUP:
        return "CUP PRESENTE IN PIU' CANDIDATURE"
    elif CUP in MISSING_TEMPLATE:
        return 'TEMPLATE MANCANTE'
    elif CUP in WRONG_TEMPLATE:
        return 'TEMPLATE ERRATO'
    else:
        return 'NESSUN PROBLEMA RISCONTRATO AL MOMENTO'

In [26]:
# applica classificazione
df_DIPE['CLASSIFICAZIONE_ISSUE_CUP']  = df_DIPE.CUP.apply(classify_CUP)

In [27]:
print(df_DIPE.CUP.count())
#crea aggregati
df_DIPE.groupby(by=['CLASSIFICAZIONE_ISSUE_CUP','STATO_CANDIDATURA']).CUP.count()

26574


CLASSIFICAZIONE_ISSUE_CUP               STATO_CANDIDATURA
CUP NON PRESENTE NEL FEEDBACK DIPE      FINANZIATA             638
                                        RINUNCIATA              90
CUP PRESENTE IN PIU' CANDIDATURE        FINANZIATA              34
                                        RINUNCIATA              32
NESSUN PROBLEMA RISCONTRATO AL MOMENTO  FINANZIATA           22948
                                        RINUNCIATA            2469
TEMPLATE ERRATO                         FINANZIATA             336
                                        RINUNCIATA              27
Name: CUP, dtype: int64

In [28]:
df_DIPE[(df_DIPE.CLASSIFICAZIONE_ISSUE_CUP == "CUP PRESENTE IN PIU' CANDIDATURE")&(df_DIPE.STATO_CANDIDATURA=='FINANZIATA')].CUP.value_counts()>1
# GIà SANATO
# eventualmente approfondire

D31F22004420006     True
F91F22002170006     True
D91F22003340006     True
J41F22002450006    False
J21F22002490006    False
J11F22002310006    False
B51F22002130006    False
B91C22001040006    False
B61C22001070006    False
D51F22002870006    False
G21F22002260006    False
C81F22002280006    False
E91F22002250006    False
G61F22002770006    False
I31F22002770006    False
B81F22002740006    False
D81F22002430006    False
F61F22002080006    False
E31C22001360006    False
D81F22002440006    False
F61F22002320006    False
J41F22002440006    False
B51F22002330006    False
B71F22002310006    False
H61C22001120006    False
H41C22000980006    False
J71C22000960006    False
C51F22002010006    False
H61C22001090006    False
I71F22002100006    False
J51F22002080006    False
Name: CUP, dtype: bool

In [29]:
df_DIPE[(df_DIPE.CLASSIFICAZIONE_ISSUE_CUP == "CUP già presente in OPEN COESIONE")].SOGGETTO_TITOLARE.to_list()
# 

[]

In [30]:
df_DIPE[df_DIPE.CLASSIFICAZIONE_ISSUE_CUP == "CUP già presente in OPEN COESIONE"]

,CODICE_CUP,AVVISO,STATO_CANDIDATURA,SOGGETTO_TITOLARE,DATA_GENERAZIONE,TEMPLATE,COPERTURA_FINANZIARIA,COSTO,FINANZIAMENTO,DESCRIZIONE_AGGREGATA,NATURA,TIPOLOGIA,SETTORE,SOTTOSETTORE,CATEGORIA,STATO_PROGETTO,CUP,CLASSIFICAZIONE_ISSUE_CUP


In [31]:
df_DIPE[(df_DIPE.CLASSIFICAZIONE_ISSUE_CUP=='CUP NON PRESENTE NEL FEEDBACK DIPE')&(df_DIPE.STATO_CANDIDATURA!='RINUNCIATA')]

,CODICE_CUP,AVVISO,STATO_CANDIDATURA,SOGGETTO_TITOLARE,DATA_GENERAZIONE,TEMPLATE,COPERTURA_FINANZIARIA,COSTO,FINANZIAMENTO,DESCRIZIONE_AGGREGATA,NATURA,TIPOLOGIA,SETTORE,SOTTOSETTORE,CATEGORIA,STATO_PROGETTO,CUP,CLASSIFICAZIONE_ISSUE_CUP
84,C71C22001500001,"1.4.5 ""Piattaforma Notifiche Digitali” Comuni ...",FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,C71C22001500001,CUP NON PRESENTE NEL FEEDBACK DIPE
159,E81C22001890001,1.4.1 Esperienza del Cittadino - Scuole - Giug...,FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,E81C22001890001,CUP NON PRESENTE NEL FEEDBACK DIPE
170,D21C22001610005,1.3.1 Piattaforma Digitale Nazionale Dati - Co...,FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,D21C22001610005,CUP NON PRESENTE NEL FEEDBACK DIPE
176,D11J22000160001,1.4.1 Esperienza del Cittadino - Scuole - Giug...,FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,D11J22000160001,CUP NON PRESENTE NEL FEEDBACK DIPE
182,E81C22001880001,1.2 Abilitazione al Cloud - Scuole - Giugno 2022,FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,E81C22001880001,CUP NON PRESENTE NEL FEEDBACK DIPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26232,E21H22000060005,1.4.4 Adozione SPID CIE-Amministrazioni divers...,FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,E21H22000060005,CUP NON PRESENTE NEL FEEDBACK DIPE
26267,D21H22000120005,1.4.4 Adozione SPID CIE-Amministrazioni divers...,FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,D21H22000120005,CUP NON PRESENTE NEL FEEDBACK DIPE
26306,I11J22000090006,1.4.4 Adozione SPID CIE - Comuni - Settembre ...,FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,I11J22000090006,CUP NON PRESENTE NEL FEEDBACK DIPE
26317,H77H22002430001,1.4.3 - app IO - Altri Enti - Maggio 2022,FINANZIATA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CUP INESISTENTE,H77H22002430001,CUP NON PRESENTE NEL FEEDBACK DIPE


In [34]:
df[df['CODICE_CUP']=='C71C22001500001']

,CODICE_CUP,SOGGETTO_TITOLARE,DATA_GENERAZIONE,TEMPLATE,COPERTURA_FINANZIARIA,COSTO,FINANZIAMENTO,DESCRIZIONE_AGGREGATA,NATURA,TIPOLOGIA,SETTORE,SOTTOSETTORE,CATEGORIA,STATO_PROGETTO


In [36]:
df_DIPE.to_csv('../data/02_primary/CUP_analysis.csv', sep=";", quoting=1, index=False)